In [ ]:
1.图片预处理，洗数据
2.将train每个图片路径以列表形式保存，分成训练集和测试集。
3.创建tfrecord。分别将训练集和测试集的图片放入tfrecord中，得到训练tfrecord和测试tfrecord。
4.建立模型。三卷积两全连接加最后一层输出层。
5.训练模型。从tfrecord采用minibatch方式读取训练。
6.模型保存。
7.模型读取，进行单个图片的识别。

#### 训练图片全部在一个文件夹train，再分别放到train1的两个cat和dog文件夹

In [ ]:
import os  
import tensorflow as tf  
from PIL import Image
# os.getcwd()
# os.makedirs('train1/train_cat')
# os.makedirs('train1/train_dog')

import os  
import tensorflow as tf  
import shutil,os
from PIL import Image

for img_name in os.listdir('train'):
    if(img_name[0:3]=='cat'):
        shutil.move('train/'+img_name,'train1/train_cat')
for img_name in os.listdir('train'):
    if(img_name[0:3]=='dog'):
        shutil.move('train/'+img_name,'train1/train_dog')

#### 将数据路径放到一个list，打乱顺序，分为训练集和测试集，建立两个tfrecord 

In [28]:
filenameTrain = 'dog_cat_train.tfrecords'
filenameTest = 'dog_cat_test.tfrecords'
orig_picture = 'train1'
#数据目录 train1/dog/,train1/cat
random.seed(2)
tfresizeW = 128#resize图片大小
tfresizeH = 128
def creat_tfrecord(test_ratio):
    totalimg = list()
    totallabel = list()
    writerTrain = tf.python_io.TFRecordWriter(filenameTrain)
    writerTest = tf.python_io.TFRecordWriter(filenameTest)
    for index,name in enumerate(classes):
        class_path = orig_picture +"/"+ name+"/" 
        for img_name in os.listdir(class_path):
            img_path = class_path + img_name
            totalimg.append(img_path)
            totallabel.append(index)
    zipList = list(zip(totalimg,totallabel))
    random.shuffle(zipList)
    totalimg[:],totallabel[:] = zip(*zipList)
    i = 0
    for imgPath,imgLable in zip(totalimg,totallabel):
        img = Image.open(imgPath)
        img = img.resize((tfresizeW,tfresizeH))
        img_raw = img.tobytes()
        example = tf.train.Example(  #将数据写入到数据缓冲区
           features=tf.train.Features(feature={  
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[imgLable])),  
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))  
           }))
        if i<=len(totalimg) * test_ratio:
            writerTest.write(example.SerializeToString())
        else:
            writerTrain.write(example.SerializeToString())
        i += 1
    writerTrain.close()
    writerTest.close()
    
creat_tfrecord(0.2)

In [1]:
import os  
import tensorflow as tf  
from PIL import Image
# os.getcwd()
# os.makedirs('train/train_cat')
# os.makedirs('train/train_dog')

import os  
import tensorflow as tf  
import shutil,os
from PIL import Image
import numpy as np

#### 读取tfrecord

In [2]:
def read_and_decode(filename):  #读取队列，先进先出，读取一个之后，后面自动补齐
    # 创建文件队列,不限读取的数量  
    filename_queue = tf.train.string_input_producer([filename]) #读取解析队列 
    # create a reader from file queue  
    reader = tf.TFRecordReader()  
    # reader从文件队列中读入一个序列化的样本  
    _, serialized_example = reader.read(filename_queue)  #返回文件名和文件
    # get feature from serialized example  
    # 解析符号化的样本  
    features = tf.parse_single_example(  #将example协议块解析为张量
        serialized_example,  
        features={  
            'label': tf.FixedLenFeature([], tf.int64),  
            'img_raw': tf.FixedLenFeature([], tf.string)  
        })  
    label = features['label']  
    img = features['img_raw']  
    img = tf.decode_raw(img, tf.uint8)  #将to_bytes的字符串格式转成uint8
    img = tf.reshape(img, [128, 128, 3])  
#     img = tf.cast(img, tf.float32) *(1./255) - 0.5
    img = tf.image.per_image_standardization(img)
    label = tf.cast(label, tf.int32)  
    return img, label 

In [5]:
NUM_CHANNELS = 3

CONV1_DEEP = 32
CONV1_SIZE = 3

CONV2_DEEP = 64
CONV2_SIZE = 3

FC_SIZE1 = 128
FC_SIZE2 = 256

REGULARIZATION_RATE = 0.0001

In [2]:
#写网络用于训练，并存储网络
#=========================================================================
import tensorflow as tf
#=========================================================================
#网络结构定义
    #输入参数：images，image batch、4D tensor、tf.float32、[batch_size, width, height, channels]
    #返回参数：logits, float、 [batch_size, n_classes]
def inference(images, batch_size, n_classes,train,regularizer_num):
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
#一个简单的卷积神经网络，卷积+池化层x2，全连接层x2，最后一个softmax层做分类。
#卷积层1
#64个3x3的卷积核（3通道），padding=’SAME’，表示padding后卷积的图与原图尺寸一致，激活函数relu()
    with tf.variable_scope('conv1') as scope:

        weights1 = tf.Variable(tf.truncated_normal(shape=[CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP], 
                                                   stddev = 0.01, dtype = tf.float32), 
                              name = 'weights', dtype = tf.float32)

        biases1 = tf.Variable(tf.constant(value = 0, dtype = tf.float32, shape = [CONV1_DEEP]),
                             name = 'biases', dtype = tf.float32)

        conv1 = tf.nn.conv2d(images, weights1, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv1, biases1)
        conv11 = tf.nn.relu(pre_activation, name= scope.name)

#池化层1
#3x3最大池化，步长strides为2，池化后执行lrn()操作，局部响应归一化，对训练有利。
    with tf.variable_scope('pooling1_lrn') as scope:
        pool1 = tf.nn.max_pool(conv11, ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME', name='pooling1')
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm1')
        norm1 =pool1
#卷积层2
#16个3x3的卷积核（16通道），padding=’SAME’，表示padding后卷积的图与原图尺寸一致，激活函数relu()
    with tf.variable_scope('conv2') as scope:
        weights2 = tf.Variable(tf.truncated_normal(shape=[CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP], 
                                                   stddev = 0.01, dtype = tf.float32), 
                              name = 'weights', dtype = tf.float32)

        biases2 = tf.Variable(tf.constant(value = 0, dtype = tf.float32, shape = [CONV2_DEEP]),
                             name = 'biases', dtype = tf.float32)

        conv2 = tf.nn.conv2d(norm1, weights2, strides = [1,1,1,1],padding='SAME')
        pre_activation = tf.nn.bias_add(conv2, biases2)
        conv22 = tf.nn.relu(pre_activation, name='conv2')
 #池化层2
#3x3最大池化，步长strides为2，池化后执行lrn()操作，
    #pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        pool2 = tf.nn.max_pool(conv22, ksize=[1,3,3,1], strides=[1,1,1,1],padding='SAME',name='pooling2')
        norm2 = tf.nn.lrn(pool2, depth_radius=4, bias=1.0, alpha=0.001/9.0,beta=0.75,name='norm2')
        norm2 =pool2
# #卷积层3
# #16个3x3的卷积核（16通道），padding=’SAME’，表示padding后卷积的图与原图尺寸一致，激活函数relu()
#     with tf.variable_scope('conv3') as scope:
#         weights5 = tf.Variable(tf.truncated_normal(shape=[CONV3_SIZE,CONV3_SIZE,CONV2_DEEP,CONV3_DEEP], 
#                                                    stddev = 0.01, dtype = tf.float32), 
#                               name = 'weights', dtype = tf.float32)

#         biases5 = tf.Variable(tf.constant(value = 0, dtype = tf.float32, shape = [CONV3_DEEP]),
#                              name = 'biases', dtype = tf.float32)

#         conv = tf.nn.conv2d(norm2, weights5, strides = [1,1,1,1],padding='SAME')
#         pre_activation = tf.nn.bias_add(conv, biases5)
#         conv5 = tf.nn.relu(pre_activation, name='conv5')
#  #池化层3
# #3x3最大池化，步长strides为2，池化后执行lrn()操作，
#     #pool2 and norm2
#     with tf.variable_scope('pooling5_lrn') as scope:
#         pool5 = tf.nn.max_pool(conv5, ksize=[1,3,3,1], strides=[1,1,1,1],padding='SAME',name='pooling5')
#         norm5 = tf.nn.lrn(pool5, depth_radius=4, bias=1.0, alpha=0.001/9.0,beta=0.75,name='norm5')

    
#全连接层3
#128个神经元，将之前pool层的输出reshape成一行，激活函数relu()
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(norm2, shape=[batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights3 = tf.Variable(tf.truncated_normal(shape=[dim,FC_SIZE1], stddev = 0.01, dtype = tf.float32),
                             name = 'weights', dtype = tf.float32)
        wloss_w3 = regularizer(weights3)
        if regularizer_num is not None:
            tf.add_to_collection('losses', wloss_w3)
        biases3 = tf.Variable(tf.constant(value = 0, dtype = tf.float32, shape = [FC_SIZE1]), 
                             name = 'biases', dtype=tf.float32)

        local3 = tf.nn.relu(tf.matmul(reshape, weights3) + biases3, name=scope.name)
        #dropout层        
#     with tf.variable_scope('dropout') as scope:
#         if train is not None:
#             local3 = tf.nn.dropout(local3, 0.5)

#全连接层4
#128个神经元，激活函数relu() 
    with tf.variable_scope('local4') as scope:
        weights4 = tf.Variable(tf.truncated_normal(shape=[FC_SIZE1,FC_SIZE2], stddev = 0.01, 
                                                   dtype = tf.float32),name = 'weights',dtype = tf.float32)
        wloss_w4 = regularizer(weights4)
        if regularizer_num is not None:
            tf.add_to_collection('losses', wloss_w4)
        biases4 = tf.Variable(tf.constant(value = 0, dtype = tf.float32, shape = [FC_SIZE2]),
                             name = 'biases', dtype = tf.float32)

        local4 = tf.nn.relu(tf.matmul(local3, weights4) + biases4, name='local4')
#dropout层        
    with tf.variable_scope('dropout') as scope:
        if train is not None:
            local4 = tf.nn.dropout(local4, 0.9)


#Softmax回归层
#将前面的FC层输出，做一个线性回归，计算出每一类的得分，在这里是2类，所以这个层输出的是两个得分。
    with tf.variable_scope('softmax_linear') as scope:
        weights5 = tf.Variable(tf.truncated_normal(shape=[FC_SIZE2, n_classes], stddev = 0.01, 
                                                   dtype = tf.float32),name = 'softmax_linear', dtype = tf.float32)

        biases5 = tf.Variable(tf.constant(value = 0, dtype = tf.float32, shape = [n_classes]),
                             name = 'biases', dtype = tf.float32)

        softmax_linear = tf.add(tf.matmul(local4, weights5), biases5, name='softmax_linear')
#         softmax_linear = tf.nn.softmax(softmax_linear)
    return softmax_linear

In [5]:
def losses(logits, labels,flag):
    with tf.variable_scope('loss') as scope:
        cross_entropy =tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        tf.add_to_collection('losses', loss)
        if flag==1:
            all_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')
        else:
            all_loss = loss
        tf.summary.scalar(scope.name+'/loss', all_loss)
    return all_loss

In [6]:
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op

In [7]:
def evaluation(logits, labels):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        tf.summary.scalar(scope.name+'/accuracy', accuracy)
    return accuracy

In [4]:
#变量声明
N_CLASSES = 2  #分类数量
IMG_W = 128   # resize图像，太大的话训练时间久
IMG_H = 128
CHANNEL = 3
BATCH_SIZE =32#训练的mini_batch大小
CAPACITY = 2000#队列存放数量
MAX_STEP = 10000 # 最大训练步骤
learning_rate = 0.0001# 一般小于0.0001初始学习率
logs_train_dir = 'log'    #logs存储路径

### 将train和test分开，但共用一个网络结构，不能建立新的test否则输入重新生成一个分支，不能使用最新的参数。 

In [9]:
is_training = tf.placeholder(tf.bool, shape=())

train_batch = tf.placeholder(tf.float32,shape=(BATCH_SIZE,IMG_W,IMG_H,CHANNEL))
train_label_batch = tf.placeholder(tf.int64,shape=(BATCH_SIZE))

val_batch = tf.placeholder(tf.float32,shape=(BATCH_SIZE,IMG_W,IMG_H,CHANNEL))
val_label_batch = tf.placeholder(tf.int64,shape=(BATCH_SIZE))

x = tf.cond(is_training, lambda: train_batch, lambda: val_batch)
y = tf.cond(is_training, lambda: train_label_batch, lambda: val_label_batch)
isdrop = tf.cond(is_training, lambda: True, lambda: False)
isl2 = tf.cond(is_training, lambda: True, lambda: False)

logits = inference(x,BATCH_SIZE,N_CLASSES,isdrop,isl2)
loss = losses(logits,y,isl2)
train_op = trainning(loss,learning_rate)
acc = evaluation(logits, y)


#这个是log汇总记录
summary_op = tf.summary.merge_all() 

In [10]:
train_img,train_label = read_and_decode('dog_cat_train.tfrecords')
test_img,test_label = read_and_decode('dog_cat_test.tfrecords')
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())   

In [11]:
img_batch, label_batch = tf.train.shuffle_batch([train_img, train_label],
                                                batch_size=BATCH_SIZE, capacity=CAPACITY,
                                                min_after_dequeue=1000)
img_batch_test,label_batch_test = tf.train.shuffle_batch([test_img, test_label],
                                                batch_size=BATCH_SIZE, capacity=CAPACITY,
                                                min_after_dequeue=1000)

In [12]:
with tf.Session() as sess:
    
    train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph) 
    saver = tf.train.Saver()
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for step in np.arange(MAX_STEP):
        if coord.should_stop():  #线程被要求停止则返回True
            break   
        img_batch1,label_batch1=sess.run([img_batch,label_batch])
        img_batch_test1,label_batch_test1=sess.run([img_batch_test,label_batch_test])
        sess.run(train_op,feed_dict={is_training: True,train_batch:img_batch1,train_label_batch:label_batch1,
                                    val_batch:img_batch_test1,val_label_batch:label_batch_test1})
        if step % 50  == 0:
            tra_loss, tra_acc = sess.run([loss,acc],feed_dict={is_training:True,train_batch:img_batch1,train_label_batch:label_batch1,
                                                              val_batch:img_batch_test1,val_label_batch:label_batch_test1})
            tes_loss, tes_acc = sess.run([loss,acc],feed_dict={is_training:False,train_batch:img_batch1,train_label_batch:label_batch1,
                                                              val_batch:img_batch_test1,val_label_batch:label_batch_test1})
            print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
            print('Step %d, test loss = %.2f, test accuracy = %.2f%%' %(step, tes_loss, tes_acc*100.0))
            
            summary_str = sess.run(summary_op,feed_dict={is_training:False,train_batch:img_batch1,train_label_batch:label_batch1,
                                                              val_batch:img_batch_test1,val_label_batch:label_batch_test1})
            train_writer.add_summary(summary_str, step)
        if (step + 1) == MAX_STEP:
            checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=step)
#     tes_loss, tes_acc = sess.run([test_loss,test_acc])
#     print('Step %d, test loss = %.2f, test accuracy = %.2f%%' %(step, tes_loss, tes_acc*100.0))
    coord.request_stop()#请求该线程停止
    

Step 0, train loss = 0.69, train accuracy = 56.25%
Step 0, test loss = 0.69, test accuracy = 50.00%
Step 50, train loss = 0.68, train accuracy = 53.12%
Step 50, test loss = 0.68, test accuracy = 50.00%
Step 100, train loss = 0.67, train accuracy = 62.50%
Step 100, test loss = 0.68, test accuracy = 50.00%
Step 150, train loss = 0.60, train accuracy = 65.62%
Step 150, test loss = 0.58, test accuracy = 81.25%
Step 200, train loss = 0.54, train accuracy = 68.75%
Step 200, test loss = 0.50, test accuracy = 78.12%
Step 250, train loss = 0.64, train accuracy = 65.62%
Step 250, test loss = 0.57, test accuracy = 65.62%
Step 300, train loss = 0.64, train accuracy = 68.75%
Step 300, test loss = 0.61, test accuracy = 56.25%
Step 350, train loss = 0.62, train accuracy = 62.50%
Step 350, test loss = 0.52, test accuracy = 81.25%
Step 400, train loss = 0.63, train accuracy = 62.50%
Step 400, test loss = 0.63, test accuracy = 53.12%
Step 450, train loss = 0.47, train accuracy = 71.88%
Step 450, test lo

Step 3900, train loss = 0.41, train accuracy = 84.38%
Step 3900, test loss = 0.27, test accuracy = 90.62%
Step 3950, train loss = 0.18, train accuracy = 93.75%
Step 3950, test loss = 0.40, test accuracy = 87.50%
Step 4000, train loss = 0.28, train accuracy = 87.50%
Step 4000, test loss = 0.29, test accuracy = 87.50%
Step 4050, train loss = 0.34, train accuracy = 84.38%
Step 4050, test loss = 0.51, test accuracy = 75.00%
Step 4100, train loss = 0.15, train accuracy = 96.88%
Step 4100, test loss = 0.46, test accuracy = 78.12%
Step 4150, train loss = 0.28, train accuracy = 81.25%
Step 4150, test loss = 0.37, test accuracy = 84.38%
Step 4200, train loss = 0.29, train accuracy = 90.62%
Step 4200, test loss = 0.33, test accuracy = 81.25%
Step 4250, train loss = 0.41, train accuracy = 81.25%
Step 4250, test loss = 0.49, test accuracy = 75.00%
Step 4300, train loss = 0.34, train accuracy = 84.38%
Step 4300, test loss = 0.26, test accuracy = 87.50%
Step 4350, train loss = 0.14, train accuracy =

Step 7800, train loss = 0.03, train accuracy = 100.00%
Step 7800, test loss = 0.58, test accuracy = 84.38%
Step 7850, train loss = 0.01, train accuracy = 100.00%
Step 7850, test loss = 0.85, test accuracy = 78.12%
Step 7900, train loss = 0.01, train accuracy = 100.00%
Step 7900, test loss = 1.47, test accuracy = 71.88%
Step 7950, train loss = 0.02, train accuracy = 100.00%
Step 7950, test loss = 0.96, test accuracy = 78.12%
Step 8000, train loss = 0.01, train accuracy = 100.00%
Step 8000, test loss = 0.64, test accuracy = 87.50%
Step 8050, train loss = 0.01, train accuracy = 100.00%
Step 8050, test loss = 0.96, test accuracy = 87.50%
Step 8100, train loss = 0.00, train accuracy = 100.00%
Step 8100, test loss = 0.67, test accuracy = 84.38%
Step 8150, train loss = 0.01, train accuracy = 100.00%
Step 8150, test loss = 1.50, test accuracy = 75.00%
Step 8200, train loss = 0.01, train accuracy = 100.00%
Step 8200, test loss = 1.10, test accuracy = 78.12%
Step 8250, train loss = 0.01, train a

### 一次读一张图片，添加到列表中，由于内存有限，所以分几次读取 

In [6]:
L=[]
#七.读取一张图，用于识别
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#获取一张图片
def get_one_image(train):
    #输入参数：train,训练图片的路径
    #返回参数：image，从训练图片中随机抽取一张图片
    img_dir = train   #识别图片的路径

    img = Image.open(img_dir)
    plt.imshow(img)
    imag = img.resize([128, 128])  #由于图片在预处理阶段以及resize，因此该命令可略
    image = np.array(imag)
    return image


#八.测试图片

def evaluate_one_image():
    
    with tf.Graph().as_default():
        BATCH_SIZE = 1#识别一张图片
        N_CLASSES = 2#类别
        img = tf.placeholder(tf.float32,shape=[128,128,3])
        image1 = tf.cast(img, tf.float32)
        image2 = tf.image.resize_image_with_crop_or_pad(image1, 128, 128)
        image3 = tf.image.per_image_standardization(image2)
        image4 = tf.reshape(image3, [1,128, 128, 3])

        logit1 = inference(image4, BATCH_SIZE, N_CLASSES,0,0)

        logit = tf.nn.softmax(logit1)


        # you need to change the directories to yours.
        logs_train_dir = 'log' 

        saver = tf.train.Saver()

        with tf.Session() as sess:

            ckpt = tf.train.get_checkpoint_state(logs_train_dir)

            if ckpt and ckpt.model_checkpoint_path:

                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]

                saver.restore(sess, ckpt.model_checkpoint_path)
#                 print('Loading success, global_step is %s' % global_step)
#             else:
#                 print('No checkpoint file found')
           #             saver.restore(sess,'/home/liu/new_start/make_data/123/net/my_net_rnn.ckpt')
            
#             for img_name in os.listdir('test1'):  #返回指定的文件夹包含的文件或文件夹的名字的列表

            for i in range(7001,12501):
                if i%100==0:
                    print(i)
                picture_dir = 'test1/'+str(i)+'.jpg'
                img1 = get_one_image(picture_dir)
        
                prediction = sess.run(logit, feed_dict={img:img1})
#                 print(prediction)
                max_index = np.argmax(prediction)
#                 print(max_index)
                if max_index==1:
#                 print('This is a dog with possibility %.6f' %prediction[:, 1])
                    L.append(prediction[0][1])
#             elif max_index==1:
#                            print('This is a jiwawa with possibility %.6f' %prediction[:, 1])
#             elif max_index==2:
#                            print('This is a poodle with possibility %.6f' %prediction[:, 2])
                else:
#                 print('This is a cat with possibility %.6f' %prediction[:, 0])
                    L.append(prediction[0][1])


if __name__ == '__main__':
#     train_dir = '/home/liu/new_start/test/inputdata'
#     picture_dir = 'test1/20.jpg'
#     picture_dir = '11111.jpeg'
#     train, train_label, val, val_label = get_files(train_dir, 0.1)
#     img = get_one_image(picture_dir)  #picture_dir为被测试图片的路径
    evaluate_one_image()
    

INFO:tensorflow:Restoring parameters from log/model.ckpt-9999


FileNotFoundError: [Errno 2] No such file or directory: 'test1/7001.jpg'

In [ ]:
file = open('7001-12500.txt','w');

file.write(str(L));

file.close();

### 用于测试单张图片 

In [ ]:
if __name__ == '__main__':
#     train_dir = '/home/liu/new_start/test/inputdata'
    picture_dir = 'test1/48.jpg'
#     picture_dir = '11111.jpeg'
#     train, train_label, val, val_label = get_files(train_dir, 0.1)
    img = get_one_image(picture_dir)  #picture_dir为被测试图片的路径
    evaluate_one_image(img)